In [1]:
# #This  program is created by rakesh from crddmirror system on Tue 01 Dec 2020 11:22:29 PM IST.
#License: Attribution-NonCommercial-ShareAlike 4.0 International. For feedback, please email at rakesh4osdd@gmail.com
# check for updated code jupyter file name 'virhostnet' in MITAB 2.5 format
import csv
import re
import fileinput

In [2]:
# define input and output file with taxid mapping file
input1='virhostnet-query.txt'
output1='hpi-'+input1
both2='both-'+input1
pathogen3='pathogen-'+input1
mapping_taxid='map-taxid-246-pathogen.virus-virhostnet.tsv'

In [3]:
# set output file for writing
#fieldnames = [ 'SN', 'protein1', 'protein2', 'host', 'partner']
fieldnames = [ 'host_protein1', 'pathogen_protein2', 'host', 'pathogen']
csvfile1=open(output1, 'w', newline='')
#fieldnames = ['SN', 'protein1', 'protein2', 'host', 'partner']
output = csv.DictWriter(csvfile1,fieldnames=fieldnames, delimiter='\t')
output.writeheader()

csvfile2=open(both2, 'w', newline='')
#fieldnames = ['SN', 'protein1', 'protein2', 'host', 'partner']
both = csv.DictWriter(csvfile2,fieldnames=fieldnames, delimiter='\t')
both.writeheader()

csvfile3=open(pathogen3, 'w', newline='')
#fieldnames = ['SN', 'protein1', 'protein2', 'host', 'partner']
path = csv.DictWriter(csvfile3,fieldnames=fieldnames, delimiter='\t')
path.writeheader()
#path.writerow({'SN': '1', 'protein1': 'protein1', 'protein2': 'protein2' , 'host': 'host', 'partner': 'partner2'})

47

In [4]:
def update_taxid(path_taxid):
    with fileinput.input(mapping_taxid) as f:
        for line in f:
            #print (line[:-2])
            #for line in fileinput.input(files='map2', inplace=False, mode='r'):
            taxid=line.split('\t')
            #print(taxid[0] ,taxid[1][:-2])
            #line = re.sub( taxid[0] ,taxid[1], line.rstrip())
            if re.search( rf'\b{taxid[0]}\b', path_taxid):
                new_taxid = re.sub( rf'\b{taxid[0]}\b' ,taxid[1], path_taxid)
                return str(new_taxid[:-2])
#update_taxid('taxid:694009')

In [47]:
"""def update_taxid(path_taxid):
    with fileinput.input(mapping_taxid) as f:
        for line in f:
            #print (line[:-2])
            #for line in fileinput.input(files='map2', inplace=False, mode='r'):
            taxid=line.split('\t')
            #print(taxid[0] ,taxid[1][:-2])
            #line = re.sub( taxid[0] ,taxid[1], line.rstrip())
            if re.search( taxid[0], path_taxid):
                new_taxid = re.sub( taxid[0] ,taxid[1], path_taxid)
                return str(new_taxid[:-2])
    #fileinput.nextfile()
    #fileinput.close()
#update_taxid('taxid:694009')"""

In [5]:
# code for filter MI-TAB 2.5 file
with open(input1) as csvfile:
    hpi = csv.reader(csvfile, delimiter='\t')
    #next(hpi, None)  # skip the headers
    for i, row in enumerate(hpi):
        if i >= 0:
            #remove PRO and extract ':' saparated id
            row[0]=row[0].split('-PRO_')[0].split(':')[1]
            row[1]=row[1].split('-PRO_')[0].split(':')[1]
            protein1=row[0]
            protein2=row[1]
            # search for homo sapiens 9606
            pattern1=r'\b9606\b.*'
            # search for pair
            pattern2= r"\b9606\b[^:].*\b9606\b.*"
            #print(row[2]+row[3])
            string=row[9]+'\t'+row[10]
            host=row[9]
            pathogen=row[10]
            #print(row[0], row[1], row[9],update_taxid(row[10]))
            if re.search(pattern2, string):
                partner2=pathogen
                #both.writerow({'SN': i, 'host_protein1': protein1, 'pathogen_protein2': protein2 , 'host': host, 'pathogen': partner2})
                both.writerow({'host_protein1': protein1, 'pathogen_protein2': protein2 , 'host': host, 'pathogen': partner2})
            elif re.search(pattern1, host):
                # for homogenious hame of host
                host='taxid:9606(Homo sapiens)'
                partner2= update_taxid(pathogen)
                #output.writerow({'SN': i, 'host_protein1': protein1, 'pathogen_protein2': protein2 , 'host': host, 'pathogen': partner2})
                output.writerow({'host_protein1': protein1, 'pathogen_protein2': protein2 , 'host': host, 'pathogen': partner2}) 
            elif re.search(pattern1, pathogen):
                #print(row[0], row[1], row[9],row[10])
                protein1=row[1] # interchange of protein
                protein2=row[0]
                #host=row[3]
                # for homogenious hame of host
                host='taxid:9606(Homo sapiens)'
                partner2=update_taxid(row[9])
                #output.writerow({'SN': i, 'host_protein1': protein1, 'pathogen_protein2': protein2 , 'host': host, 'pathogen': partner2})
                output.writerow({'host_protein1': protein1, 'pathogen_protein2': protein2 , 'host': host, 'pathogen': partner2})
            else:
                #print(row[0], row[1], row[9],row[10])
                partner2=pathogen
                path.writerow({'host_protein1': row[0] , 'pathogen_protein2': row[1] , 'host': row[9], 'pathogen': row[10]})
                #print("No match",row)

csvfile1.flush()
csvfile2.flush()
csvfile3.flush()

In [6]:
# check output files and stats
! tail -n2 {output1} {both2} {pathogen3}
! wc -l {output1} {both2} {pathogen3} ; wc -l {input1}
#! rm {output1} {both2} {pathogen3}
! tail -n +2 hpi-{input1} |cut -f1,2|sort -u|wc -l ; 
! tail -n +2 {input1} |cut -f1,2|sort -u|wc -l

==> hpi-virhostnet-query.txt <==
X6RLX0	NP_803144	taxid:9606(Homo sapiens)	taxid:41856(Hepatitis C virus genotype 1)
X6RLX0	NP_803144	taxid:9606(Homo sapiens)	taxid:41856(Hepatitis C virus genotype 1)

==> both-virhostnet-query.txt <==
Q9Y5Q3	Q14494	taxid:9606	taxid:9606
Q9Y5Q3	Q9Y5Q3	taxid:9606	taxid:9606

==> pathogen-virhostnet-query.txt <==
V9LLX6	V9LLX6	taxid:295027	taxid:295027
W5QKJ0	W5QKQ9	taxid:1979162	taxid:1979162
  33301 hpi-virhostnet-query.txt
    159 both-virhostnet-query.txt
   5185 pathogen-virhostnet-query.txt
  38645 total
38642 virhostnet-query.txt
30193
34307


In [7]:
csvfile1.close()
csvfile2.close()
csvfile3.close()

In [8]:
# check for Mt Protein exist in the hpi-{input1} file using Mt_Proteins_7432_working_uniprot_ids.txt and mt-protein-uniprot.txt
! head -n1 hpi-{input1} > mnDNA_hpi-{input1}
! grep -a -w -f ../Mt_Proteins_7432_working_uniprot_ids.txt hpi-{input1} >> mnDNA_hpi-{input1}
! wc -l mnDNA_hpi-{input1}; wc -l hpi-{input1}
! head -n2 mnDNA_hpi-{input1} hpi-{input1}

15157 mnDNA_hpi-virhostnet-query.txt
33301 hpi-virhostnet-query.txt
==> mnDNA_hpi-virhostnet-query.txt <==
host_protein1	pathogen_protein2	host	pathogen
Q6P587	NP_828867	taxid:9606(Homo sapiens)	taxid:694009(Severe acute respiratory syndrome-related coronavirus)

==> hpi-virhostnet-query.txt <==
host_protein1	pathogen_protein2	host	pathogen
Q6P4E1	Q32ZE1	taxid:9606(Homo sapiens)	taxid:64320(Zika virus)


In [9]:
# summay count uniq values in each column of the mt protein HPI file
c1 = ! tail -n +2 mnDNA_hpi-{input1}|cut -f1|sort -u|wc -l ; head -n1 mnDNA_hpi-{input1} |cut -f1
c2 = ! tail -n +2 mnDNA_hpi-{input1}|cut -f2|sort -u|wc -l ; head -n1 mnDNA_hpi-{input1} |cut -f2
c3 = ! tail -n +2 mnDNA_hpi-{input1}|cut -f3|sort -u|wc -l ; head -n1 mnDNA_hpi-{input1} |cut -f3
c4 = ! tail -n +2 mnDNA_hpi-{input1}|cut -f4|sort -u|wc -l ; head -n1 mnDNA_hpi-{input1} |cut -f4
c12 = ! tail -n +2 mnDNA_hpi-{input1}|cut -f1,2|sort -u|wc -l ; head -n1 mnDNA_hpi-{input1} |cut -f1,2
c_all = ! tail -n +2 mnDNA_hpi-{input1}|sort -u|wc -l ; head -n1 mnDNA_hpi-{input1}
print (c1,c2,c3,c4,c12, c_all)

['2369', 'host_protein1'] ['751', 'pathogen_protein2'] ['1', 'host'] ['189', 'pathogen'] ['14092', 'host_protein1\tpathogen_protein2'] ['14092', 'host_protein1\tpathogen_protein2\thost\tpathogen']


In [10]:
# split file into mDNA and nDNA list
! grep -a -v -w -f ../mt-protein-uniprot.txt mnDNA_hpi-{input1} > nDNA_hpi-{input1}
! head -n1 mnDNA_hpi-{input1} > mDNA_hpi-{input1}
! grep -a -w -f ../mt-protein-uniprot.txt mnDNA_hpi-{input1} >> mDNA_hpi-{input1}
! wc -l mDNA_hpi-{input1} nDNA_hpi-{input1} ; wc -l mnDNA_hpi-{input1}
! head -n3 mDNA_hpi-{input1} nDNA_hpi-{input1} mnDNA_hpi-{input1}

     91 mDNA_hpi-virhostnet-query.txt
  15067 nDNA_hpi-virhostnet-query.txt
  15158 total
15157 mnDNA_hpi-virhostnet-query.txt
==> mDNA_hpi-virhostnet-query.txt <==
host_protein1	pathogen_protein2	host	pathogen
P03905	P36801	taxid:9606(Homo sapiens)	taxid:10621(Human papillomavirus 9)
P03905	P36817	taxid:9606(Homo sapiens)	taxid:10621(Human papillomavirus 9)

==> nDNA_hpi-virhostnet-query.txt <==
host_protein1	pathogen_protein2	host	pathogen
Q6P587	NP_828867	taxid:9606(Homo sapiens)	taxid:694009(Severe acute respiratory syndrome-related coronavirus)
Q6P587	NP_828867	taxid:9606(Homo sapiens)	taxid:694009(Severe acute respiratory syndrome-related coronavirus)

==> mnDNA_hpi-virhostnet-query.txt <==
host_protein1	pathogen_protein2	host	pathogen
Q6P587	NP_828867	taxid:9606(Homo sapiens)	taxid:694009(Severe acute respiratory syndrome-related coronavirus)
Q6P587	NP_828867	taxid:9606(Homo sapiens)	taxid:694009(Severe acute respiratory syndrome-related coronavirus)


In [11]:
! head hpi-virhostnet-query.txt

In [12]:
! tail -n +2 hpi-virhostnet-query.txt|cut -f1|sort -u|wc -l
! tail -n +2 hpi-virhostnet-query.txt|cut -f2|sort -u|wc -l
! tail -n +2 hpi-virhostnet-query.txt|cut -f3|sort -u|wc -l
! tail -n +2 hpi-virhostnet-query.txt|cut -f4|sort -u|wc -l

7462
994
1
246


In [13]:
column4file='hpi-virhostnet-query.txt'
! wc -l {column4file}
for i in range(4):
    header = ! head -n1 {column4file} |cut -f{i+1}
    count = ! tail -n +2 {column4file} |cut -f{i+1}  |sort -u|wc -l
    print (header[0], count[0])

33301 hpi-virhostnet-query.txt
host_protein1 7462
pathogen_protein2 994
host 1
pathogen 246
